# Modelos de reapertura modular

### Paquetes

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import EoN
from matplotlib import rc
import scipy
import numpy as np
import random
import pandas as pd
import numpy as np
from collections import defaultdict
from pathlib import Path

### Definir parámetros epidemiológicos

##### Leer las redes para tener el tamaño

In [2]:
g        = nx.convert_node_labels_to_integers(nx.read_graphml("results/redes_eon/g_cdmx_infomap.graphml")) #red cdmx
g_susana = nx.convert_node_labels_to_integers(nx.read_graphml("results/redes_eon/g_susana.graphml"))

### Número de casos 

In [3]:
recuperados = 34849
infectados  = 10596


poblacion_real     = 9000000
poblacion_escalada = len(g.nodes())

recuperados_escalados = round(recuperados * poblacion_escalada / poblacion_real) 
infectados_escalados  = round(infectados * poblacion_escalada / poblacion_real) 

print(recuperados_escalados)
print(infectados_escalados)

#infectados_totales  = round(100*infectados_escalados/10)
#recuperados_totales = round(100*recuperados_escalados/10)

infectados_totales  = 3*infectados_escalados
recuperados_totales = 3*recuperados_escalados


#infectados_totales  = infectados_escalados
#recuperados_totales = recuperados_escalados
print(infectados_totales)
print(recuperados_totales)

28
8
24
84


#### Función para preparar la red para modelo epidemiológico

In [4]:
def funcion_preparadora(nw):

    nw = nw.copy()
    #Stochastic transition E-I 
    ei_node_attribute = {node: random.uniform(a=0.5, b=2) for node in nw.nodes()}
    #Stochastic transition I-R 
    #will multiply recovery rate for some factor so it is between 14 and 28 days 
    ir_node_attribute = {node: random.randint(a=1, b=2) for node in nw.nodes()}
    #Transmission weight - varies for each pair of interactions (some contacts are very transmisive, others not so)
    edge_attribute_dict = {edge: 0.5+random.random() for edge in nw.edges()} 
    nx.set_node_attributes(nw, values=ei_node_attribute, name='expose2infect_weight')
    nx.set_node_attributes(nw, values=ir_node_attribute, name='infect2recove_weight')
    nx.set_edge_attributes(nw, values=edge_attribute_dict, name='transmission_weight')
    
    return(nw)

##### Definir los parámetros de transmisión y recuperación

In [5]:
### rates
ei_rate = 1/1.5 # so that minimum rate is 12 hours, maximum is 72
ir_rate = 1/(14) # so minimum time is 14 days, and maximum time is 28 days


#transmission_rate = 0.155 #old SEIRSplus
#transmission_rate = 0.030
#transmission_rate = 0.00125
#transmission_rate = 0.0025
#transmission_rate = 0.0040

#this represents the baseline transmission. Let's use the one we used with SEIRSplus
#SIGMA  = 1/5.2
#GAMMA  = 1/10
GAMMA  = 1/14
#MU_I   = 0.002
R0     = 2.5
transmission_rate   = 1/(1/GAMMA) * R0

Rt = 0.995055947438355
transmission_rate   = 1/(1/GAMMA) * Rt
#transmission_rate = 0.125/2
print(transmission_rate)

0.07107542481702535


##### Definir la red de transiciones - describe la dinámica epidémica

In [6]:
#the node intrinsic transitions
H = nx.DiGraph()
H.add_node('S') #This line is actually unnecessary since 'S' does not change status intrinsically
#
H.add_edge('E', 'I', rate = ei_rate, weight_label='expose2infect_weight')
#
H.add_edge('I', 'R', rate = ir_rate, weight_label ='infect2recove_weight')

#interaction depending transitions
J = nx.DiGraph()
J.add_edge(('I', 'S'), ('I', 'E'), rate = transmission_rate, weight_label='transmission_weight')
#  The line above states that an 'I' individual will cause an 'S' individual
#  to transition to 'E' with rate equal to 0.1 times the partnership's attribute
#  'transmission_weight'.


###### Define initial states

In [7]:
IC = defaultdict(lambda: 'S')
for node in range(infectados_totales-1):

    IC[node] = 'I'
    
for node in range(infectados_totales, recuperados_totales):
    
    IC[node] = 'R'
    
return_statuses = ('S', 'E', 'I', 'R')

### Modelo con Reapertura Total

In [8]:
np.random.seed(725)
random.seed(725)
prep_g = funcion_preparadora(g)

nomen = "reapertura_total"
pathnomen = "results/" + nomen
Path(pathnomen).mkdir(parents=True, exist_ok=True)


for i in range(0, 100):
    print(i)
    np.random.seed(i)
    random.seed(i)
    outnam = pathnomen + "/" + nomen + "_" + str(i) + ".csv"
    print(outnam)
    mod_nuloAbs = EoN.Gillespie_simple_contagion(G=prep_g, 
                                            spontaneous_transition_graph=H,
                                            nbr_induced_transition_graph=J, 
                                            IC=IC, 
                                            return_statuses=return_statuses,
                                            return_full_data=True,
                                            tmax=1000
                                            )
    my_result = mod_nuloAbs
    my_result.summary()[0]
    predata = my_result.summary()[1]
    predata["t"] = my_result.summary()[0]
    df = pd.DataFrame(predata)
    df = df.assign(I_pc=100*df['I']/poblacion_escalada)
    df = df.assign(R_pc=100*df['R']/poblacion_escalada)
    df.to_csv(outnam)

    

0
results/reapertura_total/reapertura_total_0.csv
1
results/reapertura_total/reapertura_total_1.csv
2
results/reapertura_total/reapertura_total_2.csv
3
results/reapertura_total/reapertura_total_3.csv
4
results/reapertura_total/reapertura_total_4.csv
5
results/reapertura_total/reapertura_total_5.csv
6
results/reapertura_total/reapertura_total_6.csv
7
results/reapertura_total/reapertura_total_7.csv
8
results/reapertura_total/reapertura_total_8.csv
9
results/reapertura_total/reapertura_total_9.csv
10
results/reapertura_total/reapertura_total_10.csv
11
results/reapertura_total/reapertura_total_11.csv
12
results/reapertura_total/reapertura_total_12.csv
13
results/reapertura_total/reapertura_total_13.csv
14
results/reapertura_total/reapertura_total_14.csv
15
results/reapertura_total/reapertura_total_15.csv
16
results/reapertura_total/reapertura_total_16.csv
17
results/reapertura_total/reapertura_total_17.csv
18
results/reapertura_total/reapertura_total_18.csv
19
results/reapertura_total/reap

### Modelo Con Parámetros de JNSD

In [9]:
np.random.seed(725)
random.seed(725)
prep_g = funcion_preparadora(g_susana)

nomen = "jnsd"
pathnomen = "results/" + nomen
Path(pathnomen).mkdir(parents=True, exist_ok=True)


for i in range(0, 100):
    print(i)
    np.random.seed(i)
    random.seed(i)
    outnam = pathnomen + "/" + nomen + "_" + str(i) + ".csv"
    print(outnam)
    mod_nuloAbs = EoN.Gillespie_simple_contagion(G=prep_g, 
                                            spontaneous_transition_graph=H,
                                            nbr_induced_transition_graph=J, 
                                            IC=IC, 
                                            return_statuses=return_statuses,
                                            return_full_data=True,
                                            tmax=1000
                                            )
    my_result = mod_nuloAbs
    my_result.summary()[0]
    predata = my_result.summary()[1]
    predata["t"] = my_result.summary()[0]
    df = pd.DataFrame(predata)
    df = df.assign(I_pc=100*df['I']/poblacion_escalada)
    df = df.assign(R_pc=100*df['R']/poblacion_escalada)
    df.to_csv(outnam)

    

0
results/jnsd/jnsd_0.csv
1
results/jnsd/jnsd_1.csv
2
results/jnsd/jnsd_2.csv
3
results/jnsd/jnsd_3.csv
4
results/jnsd/jnsd_4.csv
5
results/jnsd/jnsd_5.csv
6
results/jnsd/jnsd_6.csv
7
results/jnsd/jnsd_7.csv
8
results/jnsd/jnsd_8.csv
9
results/jnsd/jnsd_9.csv
10
results/jnsd/jnsd_10.csv
11
results/jnsd/jnsd_11.csv
12
results/jnsd/jnsd_12.csv
13
results/jnsd/jnsd_13.csv
14
results/jnsd/jnsd_14.csv
15
results/jnsd/jnsd_15.csv
16
results/jnsd/jnsd_16.csv
17
results/jnsd/jnsd_17.csv
18
results/jnsd/jnsd_18.csv
19
results/jnsd/jnsd_19.csv
20
results/jnsd/jnsd_20.csv
21
results/jnsd/jnsd_21.csv
22
results/jnsd/jnsd_22.csv
23
results/jnsd/jnsd_23.csv
24
results/jnsd/jnsd_24.csv
25
results/jnsd/jnsd_25.csv
26
results/jnsd/jnsd_26.csv
27
results/jnsd/jnsd_27.csv
28
results/jnsd/jnsd_28.csv
29
results/jnsd/jnsd_29.csv
30
results/jnsd/jnsd_30.csv
31
results/jnsd/jnsd_31.csv
32
results/jnsd/jnsd_32.csv
33
results/jnsd/jnsd_33.csv
34
results/jnsd/jnsd_34.csv
35
results/jnsd/jnsd_35.csv
36
results/j

## Evaluar Reapertura aleatoria, diferentes niveles de población reintegrándose

In [10]:
g_05 = nx.convert_node_labels_to_integers(nx.read_graphml("results/redes_eon/g_reactivate_05percent.graphml"))
g_10 = nx.convert_node_labels_to_integers(nx.read_graphml("results/redes_eon/g_reactivate_10percent.graphml"))
g_15 = nx.convert_node_labels_to_integers(nx.read_graphml("results/redes_eon/g_reactivate_15percent.graphml"))
g_20 = nx.convert_node_labels_to_integers(nx.read_graphml("results/redes_eon/g_reactivate_20percent.graphml"))
g_25 = nx.convert_node_labels_to_integers(nx.read_graphml("results/redes_eon/g_reactivate_25percent.graphml"))
g_50 = nx.convert_node_labels_to_integers(nx.read_graphml("results/redes_eon/g_reactivate_50percent.graphml"))

#### 5%

In [11]:
np.random.seed(725)
random.seed(725)
prep_g = funcion_preparadora(g_05)

nomen = "random_05"
pathnomen = "results/" + nomen
Path(pathnomen).mkdir(parents=True, exist_ok=True)


for i in range(0, 100):
    print(i)
    np.random.seed(i)
    random.seed(i)
    outnam = pathnomen + "/" + nomen + "_" + str(i) + ".csv"
    print(outnam)
    mod_nuloAbs = EoN.Gillespie_simple_contagion(G=prep_g, 
                                            spontaneous_transition_graph=H,
                                            nbr_induced_transition_graph=J, 
                                            IC=IC, 
                                            return_statuses=return_statuses,
                                            return_full_data=True,
                                            tmax=1000
                                            )
    my_result = mod_nuloAbs
    my_result.summary()[0]
    predata = my_result.summary()[1]
    predata["t"] = my_result.summary()[0]
    df = pd.DataFrame(predata)
    df = df.assign(I_pc=100*df['I']/poblacion_escalada)
    df = df.assign(R_pc=100*df['R']/poblacion_escalada)
    df.to_csv(outnam)

    

0
results/random_05/random_05_0.csv
1
results/random_05/random_05_1.csv
2
results/random_05/random_05_2.csv
3
results/random_05/random_05_3.csv
4
results/random_05/random_05_4.csv
5
results/random_05/random_05_5.csv
6
results/random_05/random_05_6.csv
7
results/random_05/random_05_7.csv
8
results/random_05/random_05_8.csv
9
results/random_05/random_05_9.csv
10
results/random_05/random_05_10.csv
11
results/random_05/random_05_11.csv
12
results/random_05/random_05_12.csv
13
results/random_05/random_05_13.csv
14
results/random_05/random_05_14.csv
15
results/random_05/random_05_15.csv
16
results/random_05/random_05_16.csv
17
results/random_05/random_05_17.csv
18
results/random_05/random_05_18.csv
19
results/random_05/random_05_19.csv
20
results/random_05/random_05_20.csv
21
results/random_05/random_05_21.csv
22
results/random_05/random_05_22.csv
23
results/random_05/random_05_23.csv
24
results/random_05/random_05_24.csv
25
results/random_05/random_05_25.csv
26
results/random_05/random_05_2

#### 10%

In [12]:
np.random.seed(725)
random.seed(725)
prep_g = funcion_preparadora(g_10)

nomen = "random_10"
pathnomen = "results/" + nomen
Path(pathnomen).mkdir(parents=True, exist_ok=True)


for i in range(0, 100):
    print(i)
    np.random.seed(i)
    random.seed(i)
    outnam = pathnomen + "/" + nomen + "_" + str(i) + ".csv"
    print(outnam)
    mod_nuloAbs = EoN.Gillespie_simple_contagion(G=prep_g, 
                                            spontaneous_transition_graph=H,
                                            nbr_induced_transition_graph=J, 
                                            IC=IC, 
                                            return_statuses=return_statuses,
                                            return_full_data=True,
                                            tmax=1000
                                            )
    my_result = mod_nuloAbs
    my_result.summary()[0]
    predata = my_result.summary()[1]
    predata["t"] = my_result.summary()[0]
    df = pd.DataFrame(predata)
    df = df.assign(I_pc=100*df['I']/poblacion_escalada)
    df = df.assign(R_pc=100*df['R']/poblacion_escalada)
    df.to_csv(outnam)

    

0
results/random_10/random_10_0.csv
1
results/random_10/random_10_1.csv
2
results/random_10/random_10_2.csv
3
results/random_10/random_10_3.csv
4
results/random_10/random_10_4.csv
5
results/random_10/random_10_5.csv
6
results/random_10/random_10_6.csv
7
results/random_10/random_10_7.csv
8
results/random_10/random_10_8.csv
9
results/random_10/random_10_9.csv
10
results/random_10/random_10_10.csv
11
results/random_10/random_10_11.csv
12
results/random_10/random_10_12.csv
13
results/random_10/random_10_13.csv
14
results/random_10/random_10_14.csv
15
results/random_10/random_10_15.csv
16
results/random_10/random_10_16.csv
17
results/random_10/random_10_17.csv
18
results/random_10/random_10_18.csv
19
results/random_10/random_10_19.csv
20
results/random_10/random_10_20.csv
21
results/random_10/random_10_21.csv
22
results/random_10/random_10_22.csv
23
results/random_10/random_10_23.csv
24
results/random_10/random_10_24.csv
25
results/random_10/random_10_25.csv
26
results/random_10/random_10_2

#### 15%

In [13]:
np.random.seed(725)
random.seed(725)
prep_g = funcion_preparadora(g_15)

nomen = "random_15"
pathnomen = "results/" + nomen
Path(pathnomen).mkdir(parents=True, exist_ok=True)


for i in range(0, 100):
    print(i)
    np.random.seed(i)
    random.seed(i)
    outnam = pathnomen + "/" + nomen + "_" + str(i) + ".csv"
    print(outnam)
    mod_nuloAbs = EoN.Gillespie_simple_contagion(G=prep_g, 
                                            spontaneous_transition_graph=H,
                                            nbr_induced_transition_graph=J, 
                                            IC=IC, 
                                            return_statuses=return_statuses,
                                            return_full_data=True,
                                            tmax=1000
                                            )
    my_result = mod_nuloAbs
    my_result.summary()[0]
    predata = my_result.summary()[1]
    predata["t"] = my_result.summary()[0]
    df = pd.DataFrame(predata)
    df = df.assign(I_pc=100*df['I']/poblacion_escalada)
    df = df.assign(R_pc=100*df['R']/poblacion_escalada)
    df.to_csv(outnam)

    

0
results/random_15/random_15_0.csv
1
results/random_15/random_15_1.csv
2
results/random_15/random_15_2.csv
3
results/random_15/random_15_3.csv
4
results/random_15/random_15_4.csv
5
results/random_15/random_15_5.csv
6
results/random_15/random_15_6.csv
7
results/random_15/random_15_7.csv
8
results/random_15/random_15_8.csv
9
results/random_15/random_15_9.csv
10
results/random_15/random_15_10.csv
11
results/random_15/random_15_11.csv
12
results/random_15/random_15_12.csv
13
results/random_15/random_15_13.csv
14
results/random_15/random_15_14.csv
15
results/random_15/random_15_15.csv
16
results/random_15/random_15_16.csv
17
results/random_15/random_15_17.csv
18
results/random_15/random_15_18.csv
19
results/random_15/random_15_19.csv
20
results/random_15/random_15_20.csv
21
results/random_15/random_15_21.csv
22
results/random_15/random_15_22.csv
23
results/random_15/random_15_23.csv
24
results/random_15/random_15_24.csv
25
results/random_15/random_15_25.csv
26
results/random_15/random_15_2

#### 20%

In [14]:
np.random.seed(725)
random.seed(725)
prep_g = funcion_preparadora(g_20)

nomen = "random_20"
pathnomen = "results/" + nomen
Path(pathnomen).mkdir(parents=True, exist_ok=True)


for i in range(0, 100):
    print(i)
    np.random.seed(i)
    random.seed(i)
    outnam = pathnomen + "/" + nomen + "_" + str(i) + ".csv"
    print(outnam)
    mod_nuloAbs = EoN.Gillespie_simple_contagion(G=prep_g, 
                                            spontaneous_transition_graph=H,
                                            nbr_induced_transition_graph=J, 
                                            IC=IC, 
                                            return_statuses=return_statuses,
                                            return_full_data=True,
                                            tmax=1000
                                            )
    my_result = mod_nuloAbs
    my_result.summary()[0]
    predata = my_result.summary()[1]
    predata["t"] = my_result.summary()[0]
    df = pd.DataFrame(predata)
    df = df.assign(I_pc=100*df['I']/poblacion_escalada)
    df = df.assign(R_pc=100*df['R']/poblacion_escalada)
    df.to_csv(outnam)

    

0
results/random_20/random_20_0.csv
1
results/random_20/random_20_1.csv
2
results/random_20/random_20_2.csv
3
results/random_20/random_20_3.csv
4
results/random_20/random_20_4.csv
5
results/random_20/random_20_5.csv
6
results/random_20/random_20_6.csv
7
results/random_20/random_20_7.csv
8
results/random_20/random_20_8.csv
9
results/random_20/random_20_9.csv
10
results/random_20/random_20_10.csv
11
results/random_20/random_20_11.csv
12
results/random_20/random_20_12.csv
13
results/random_20/random_20_13.csv
14
results/random_20/random_20_14.csv
15
results/random_20/random_20_15.csv
16
results/random_20/random_20_16.csv
17
results/random_20/random_20_17.csv
18
results/random_20/random_20_18.csv
19
results/random_20/random_20_19.csv
20
results/random_20/random_20_20.csv
21
results/random_20/random_20_21.csv
22
results/random_20/random_20_22.csv
23
results/random_20/random_20_23.csv
24
results/random_20/random_20_24.csv
25
results/random_20/random_20_25.csv
26
results/random_20/random_20_2

#### 25%

In [15]:
np.random.seed(725)
random.seed(725)
prep_g = funcion_preparadora(g_25)

nomen = "random_25"
pathnomen = "results/" + nomen
Path(pathnomen).mkdir(parents=True, exist_ok=True)


for i in range(0, 100):
    print(i)
    np.random.seed(i)
    random.seed(i)
    outnam = pathnomen + "/" + nomen + "_" + str(i) + ".csv"
    print(outnam)
    mod_nuloAbs = EoN.Gillespie_simple_contagion(G=prep_g, 
                                            spontaneous_transition_graph=H,
                                            nbr_induced_transition_graph=J, 
                                            IC=IC, 
                                            return_statuses=return_statuses,
                                            return_full_data=True,
                                            tmax=1000
                                            )
    my_result = mod_nuloAbs
    my_result.summary()[0]
    predata = my_result.summary()[1]
    predata["t"] = my_result.summary()[0]
    df = pd.DataFrame(predata)
    df = df.assign(I_pc=100*df['I']/poblacion_escalada)
    df = df.assign(R_pc=100*df['R']/poblacion_escalada)
    df.to_csv(outnam)

    

0
results/random_25/random_25_0.csv
1
results/random_25/random_25_1.csv
2
results/random_25/random_25_2.csv
3
results/random_25/random_25_3.csv
4
results/random_25/random_25_4.csv
5
results/random_25/random_25_5.csv
6
results/random_25/random_25_6.csv
7
results/random_25/random_25_7.csv
8
results/random_25/random_25_8.csv
9
results/random_25/random_25_9.csv
10
results/random_25/random_25_10.csv
11
results/random_25/random_25_11.csv
12
results/random_25/random_25_12.csv
13
results/random_25/random_25_13.csv
14
results/random_25/random_25_14.csv
15
results/random_25/random_25_15.csv
16
results/random_25/random_25_16.csv
17
results/random_25/random_25_17.csv
18
results/random_25/random_25_18.csv
19
results/random_25/random_25_19.csv
20
results/random_25/random_25_20.csv
21
results/random_25/random_25_21.csv
22
results/random_25/random_25_22.csv
23
results/random_25/random_25_23.csv
24
results/random_25/random_25_24.csv
25
results/random_25/random_25_25.csv
26
results/random_25/random_25_2

#### 50%

In [16]:
np.random.seed(725)
random.seed(725)
prep_g = funcion_preparadora(g_50)

nomen = "random_50"
pathnomen = "results/" + nomen
Path(pathnomen).mkdir(parents=True, exist_ok=True)


for i in range(0, 100):
    print(i)
    np.random.seed(i)
    random.seed(i)
    outnam = pathnomen + "/" + nomen + "_" + str(i) + ".csv"
    print(outnam)
    mod_nuloAbs = EoN.Gillespie_simple_contagion(G=prep_g, 
                                            spontaneous_transition_graph=H,
                                            nbr_induced_transition_graph=J, 
                                            IC=IC, 
                                            return_statuses=return_statuses,
                                            return_full_data=True,
                                            tmax=1000
                                            )
    my_result = mod_nuloAbs
    my_result.summary()[0]
    predata = my_result.summary()[1]
    predata["t"] = my_result.summary()[0]
    df = pd.DataFrame(predata)
    df = df.assign(I_pc=100*df['I']/poblacion_escalada)
    df = df.assign(R_pc=100*df['R']/poblacion_escalada)
    df.to_csv(outnam)

    

0
results/random_50/random_50_0.csv
1
results/random_50/random_50_1.csv
2
results/random_50/random_50_2.csv
3
results/random_50/random_50_3.csv
4
results/random_50/random_50_4.csv
5
results/random_50/random_50_5.csv
6
results/random_50/random_50_6.csv
7
results/random_50/random_50_7.csv
8
results/random_50/random_50_8.csv
9
results/random_50/random_50_9.csv
10
results/random_50/random_50_10.csv
11
results/random_50/random_50_11.csv
12
results/random_50/random_50_12.csv
13
results/random_50/random_50_13.csv
14
results/random_50/random_50_14.csv
15
results/random_50/random_50_15.csv
16
results/random_50/random_50_16.csv
17
results/random_50/random_50_17.csv
18
results/random_50/random_50_18.csv
19
results/random_50/random_50_19.csv
20
results/random_50/random_50_20.csv
21
results/random_50/random_50_21.csv
22
results/random_50/random_50_22.csv
23
results/random_50/random_50_23.csv
24
results/random_50/random_50_24.csv
25
results/random_50/random_50_25.csv
26
results/random_50/random_50_2

## Evaluar reapertura modular 

In [17]:
g_m1 = nx.convert_node_labels_to_integers(nx.read_graphml("results/redes_eon/g_reapertura_m1.graphml")) #2mods
g_m2 = nx.convert_node_labels_to_integers(nx.read_graphml("results/redes_eon/g_reapertura_m2.graphml")) #3mods


In [18]:
np.random.seed(725)
random.seed(725)
prep_g = funcion_preparadora(g_m1)

nomen = "m1"
pathnomen = "results/" + nomen
Path(pathnomen).mkdir(parents=True, exist_ok=True)


for i in range(0, 100):
    print(i)
    np.random.seed(i)
    random.seed(i)
    outnam = pathnomen + "/" + nomen + "_" + str(i) + ".csv"
    print(outnam)
    mod_nuloAbs = EoN.Gillespie_simple_contagion(G=prep_g, 
                                            spontaneous_transition_graph=H,
                                            nbr_induced_transition_graph=J, 
                                            IC=IC, 
                                            return_statuses=return_statuses,
                                            return_full_data=True,
                                            tmax=1000
                                            )
    my_result = mod_nuloAbs
    my_result.summary()[0]
    predata = my_result.summary()[1]
    predata["t"] = my_result.summary()[0]
    df = pd.DataFrame(predata)
    df = df.assign(I_pc=100*df['I']/poblacion_escalada)
    df = df.assign(R_pc=100*df['R']/poblacion_escalada)
    df.to_csv(outnam)

    

0
results/m1/m1_0.csv
1
results/m1/m1_1.csv
2
results/m1/m1_2.csv
3
results/m1/m1_3.csv
4
results/m1/m1_4.csv
5
results/m1/m1_5.csv
6
results/m1/m1_6.csv
7
results/m1/m1_7.csv
8
results/m1/m1_8.csv
9
results/m1/m1_9.csv
10
results/m1/m1_10.csv
11
results/m1/m1_11.csv
12
results/m1/m1_12.csv
13
results/m1/m1_13.csv
14
results/m1/m1_14.csv
15
results/m1/m1_15.csv
16
results/m1/m1_16.csv
17
results/m1/m1_17.csv
18
results/m1/m1_18.csv
19
results/m1/m1_19.csv
20
results/m1/m1_20.csv
21
results/m1/m1_21.csv
22
results/m1/m1_22.csv
23
results/m1/m1_23.csv
24
results/m1/m1_24.csv
25
results/m1/m1_25.csv
26
results/m1/m1_26.csv
27
results/m1/m1_27.csv
28
results/m1/m1_28.csv
29
results/m1/m1_29.csv
30
results/m1/m1_30.csv
31
results/m1/m1_31.csv
32
results/m1/m1_32.csv
33
results/m1/m1_33.csv
34
results/m1/m1_34.csv
35
results/m1/m1_35.csv
36
results/m1/m1_36.csv
37
results/m1/m1_37.csv
38
results/m1/m1_38.csv
39
results/m1/m1_39.csv
40
results/m1/m1_40.csv
41
results/m1/m1_41.csv
42
results/m

In [19]:
np.random.seed(725)
random.seed(725)
prep_g = funcion_preparadora(g_m2)

nomen = "m2"
pathnomen = "results/" + nomen
Path(pathnomen).mkdir(parents=True, exist_ok=True)


for i in range(0, 100):
    print(i)
    np.random.seed(i)
    random.seed(i)
    outnam = pathnomen + "/" + nomen + "_" + str(i) + ".csv"
    print(outnam)
    mod_nuloAbs = EoN.Gillespie_simple_contagion(G=prep_g, 
                                            spontaneous_transition_graph=H,
                                            nbr_induced_transition_graph=J, 
                                            IC=IC, 
                                            return_statuses=return_statuses,
                                            return_full_data=True,
                                            tmax=1000
                                            )
    my_result = mod_nuloAbs
    my_result.summary()[0]
    predata = my_result.summary()[1]
    predata["t"] = my_result.summary()[0]
    df = pd.DataFrame(predata)
    df = df.assign(I_pc=100*df['I']/poblacion_escalada)
    df = df.assign(R_pc=100*df['R']/poblacion_escalada)
    df.to_csv(outnam)

    

0
results/m2/m2_0.csv
1
results/m2/m2_1.csv
2
results/m2/m2_2.csv
3
results/m2/m2_3.csv
4
results/m2/m2_4.csv
5
results/m2/m2_5.csv
6
results/m2/m2_6.csv
7
results/m2/m2_7.csv
8
results/m2/m2_8.csv
9
results/m2/m2_9.csv
10
results/m2/m2_10.csv
11
results/m2/m2_11.csv
12
results/m2/m2_12.csv
13
results/m2/m2_13.csv
14
results/m2/m2_14.csv
15
results/m2/m2_15.csv
16
results/m2/m2_16.csv
17
results/m2/m2_17.csv
18
results/m2/m2_18.csv
19
results/m2/m2_19.csv
20
results/m2/m2_20.csv
21
results/m2/m2_21.csv
22
results/m2/m2_22.csv
23
results/m2/m2_23.csv
24
results/m2/m2_24.csv
25
results/m2/m2_25.csv
26
results/m2/m2_26.csv
27
results/m2/m2_27.csv
28
results/m2/m2_28.csv
29
results/m2/m2_29.csv
30
results/m2/m2_30.csv
31
results/m2/m2_31.csv
32
results/m2/m2_32.csv
33
results/m2/m2_33.csv
34
results/m2/m2_34.csv
35
results/m2/m2_35.csv
36
results/m2/m2_36.csv
37
results/m2/m2_37.csv
38
results/m2/m2_38.csv
39
results/m2/m2_39.csv
40
results/m2/m2_40.csv
41
results/m2/m2_41.csv
42
results/m

### Reapertura modular con mayor carga

In [20]:
g_m2x = nx.convert_node_labels_to_integers(nx.read_graphml("results/redes_eon/g_reapertura_m3.graphml")) #2X
g_m3x = nx.convert_node_labels_to_integers(nx.read_graphml("results/redes_eon/g_reapertura_m4.graphml")) #3X
g_m4x = nx.convert_node_labels_to_integers(nx.read_graphml("results/redes_eon/g_reapertura_m6.graphml")) #4X
g_m5x = nx.convert_node_labels_to_integers(nx.read_graphml("results/redes_eon/g_reapertura_m5.graphml")) #4X

In [21]:
np.random.seed(725)
random.seed(725)
prep_g = funcion_preparadora(g_m2x)

nomen = "m2X"
pathnomen = "results/" + nomen
Path(pathnomen).mkdir(parents=True, exist_ok=True)


for i in range(0, 100):
    print(i)
    np.random.seed(i)
    random.seed(i)
    outnam = pathnomen + "/" + nomen + "_" + str(i) + ".csv"
    print(outnam)
    mod_nuloAbs = EoN.Gillespie_simple_contagion(G=prep_g, 
                                            spontaneous_transition_graph=H,
                                            nbr_induced_transition_graph=J, 
                                            IC=IC, 
                                            return_statuses=return_statuses,
                                            return_full_data=True,
                                            tmax=1000
                                            )
    my_result = mod_nuloAbs
    my_result.summary()[0]
    predata = my_result.summary()[1]
    predata["t"] = my_result.summary()[0]
    df = pd.DataFrame(predata)
    df = df.assign(I_pc=100*df['I']/poblacion_escalada)
    df = df.assign(R_pc=100*df['R']/poblacion_escalada)
    df.to_csv(outnam)

    

0
results/m2X/m2X_0.csv
1
results/m2X/m2X_1.csv
2
results/m2X/m2X_2.csv
3
results/m2X/m2X_3.csv
4
results/m2X/m2X_4.csv
5
results/m2X/m2X_5.csv
6
results/m2X/m2X_6.csv
7
results/m2X/m2X_7.csv
8
results/m2X/m2X_8.csv
9
results/m2X/m2X_9.csv
10
results/m2X/m2X_10.csv
11
results/m2X/m2X_11.csv
12
results/m2X/m2X_12.csv
13
results/m2X/m2X_13.csv
14
results/m2X/m2X_14.csv
15
results/m2X/m2X_15.csv
16
results/m2X/m2X_16.csv
17
results/m2X/m2X_17.csv
18
results/m2X/m2X_18.csv
19
results/m2X/m2X_19.csv
20
results/m2X/m2X_20.csv
21
results/m2X/m2X_21.csv
22
results/m2X/m2X_22.csv
23
results/m2X/m2X_23.csv
24
results/m2X/m2X_24.csv
25
results/m2X/m2X_25.csv
26
results/m2X/m2X_26.csv
27
results/m2X/m2X_27.csv
28
results/m2X/m2X_28.csv
29
results/m2X/m2X_29.csv
30
results/m2X/m2X_30.csv
31
results/m2X/m2X_31.csv
32
results/m2X/m2X_32.csv
33
results/m2X/m2X_33.csv
34
results/m2X/m2X_34.csv
35
results/m2X/m2X_35.csv
36
results/m2X/m2X_36.csv
37
results/m2X/m2X_37.csv
38
results/m2X/m2X_38.csv
39
res

In [22]:
np.random.seed(725)
random.seed(725)
prep_g = funcion_preparadora(g_m3x)

nomen = "m3X"
pathnomen = "results/" + nomen
Path(pathnomen).mkdir(parents=True, exist_ok=True)


for i in range(0, 100):
    print(i)
    np.random.seed(i)
    random.seed(i)
    outnam = pathnomen + "/" + nomen + "_" + str(i) + ".csv"
    print(outnam)
    mod_nuloAbs = EoN.Gillespie_simple_contagion(G=prep_g, 
                                            spontaneous_transition_graph=H,
                                            nbr_induced_transition_graph=J, 
                                            IC=IC, 
                                            return_statuses=return_statuses,
                                            return_full_data=True,
                                            tmax=1000
                                            )
    my_result = mod_nuloAbs
    my_result.summary()[0]
    predata = my_result.summary()[1]
    predata["t"] = my_result.summary()[0]
    df = pd.DataFrame(predata)
    df = df.assign(I_pc=100*df['I']/poblacion_escalada)
    df = df.assign(R_pc=100*df['R']/poblacion_escalada)
    df.to_csv(outnam)

    

0
results/m3X/m3X_0.csv
1
results/m3X/m3X_1.csv
2
results/m3X/m3X_2.csv
3
results/m3X/m3X_3.csv
4
results/m3X/m3X_4.csv
5
results/m3X/m3X_5.csv
6
results/m3X/m3X_6.csv
7
results/m3X/m3X_7.csv
8
results/m3X/m3X_8.csv
9
results/m3X/m3X_9.csv
10
results/m3X/m3X_10.csv
11
results/m3X/m3X_11.csv
12
results/m3X/m3X_12.csv
13
results/m3X/m3X_13.csv
14
results/m3X/m3X_14.csv
15
results/m3X/m3X_15.csv
16
results/m3X/m3X_16.csv
17
results/m3X/m3X_17.csv
18
results/m3X/m3X_18.csv
19
results/m3X/m3X_19.csv
20
results/m3X/m3X_20.csv
21
results/m3X/m3X_21.csv
22
results/m3X/m3X_22.csv
23
results/m3X/m3X_23.csv
24
results/m3X/m3X_24.csv
25
results/m3X/m3X_25.csv
26
results/m3X/m3X_26.csv
27
results/m3X/m3X_27.csv
28
results/m3X/m3X_28.csv
29
results/m3X/m3X_29.csv
30
results/m3X/m3X_30.csv
31
results/m3X/m3X_31.csv
32
results/m3X/m3X_32.csv
33
results/m3X/m3X_33.csv
34
results/m3X/m3X_34.csv
35
results/m3X/m3X_35.csv
36
results/m3X/m3X_36.csv
37
results/m3X/m3X_37.csv
38
results/m3X/m3X_38.csv
39
res

In [23]:
np.random.seed(725)
random.seed(725)
prep_g = funcion_preparadora(g_m4x)

nomen = "m4X"
pathnomen = "results/" + nomen
Path(pathnomen).mkdir(parents=True, exist_ok=True)


for i in range(0, 100):
    print(i)
    np.random.seed(i)
    random.seed(i)
    outnam = pathnomen + "/" + nomen + "_" + str(i) + ".csv"
    print(outnam)
    mod_nuloAbs = EoN.Gillespie_simple_contagion(G=prep_g, 
                                            spontaneous_transition_graph=H,
                                            nbr_induced_transition_graph=J, 
                                            IC=IC, 
                                            return_statuses=return_statuses,
                                            return_full_data=True,
                                            tmax=1000
                                            )
    my_result = mod_nuloAbs
    my_result.summary()[0]
    predata = my_result.summary()[1]
    predata["t"] = my_result.summary()[0]
    df = pd.DataFrame(predata)
    df = df.assign(I_pc=100*df['I']/poblacion_escalada)
    df = df.assign(R_pc=100*df['R']/poblacion_escalada)
    df.to_csv(outnam)

    

0
results/m4X/m4X_0.csv
1
results/m4X/m4X_1.csv
2
results/m4X/m4X_2.csv
3
results/m4X/m4X_3.csv
4
results/m4X/m4X_4.csv
5
results/m4X/m4X_5.csv
6
results/m4X/m4X_6.csv
7
results/m4X/m4X_7.csv
8
results/m4X/m4X_8.csv
9
results/m4X/m4X_9.csv
10
results/m4X/m4X_10.csv
11
results/m4X/m4X_11.csv
12
results/m4X/m4X_12.csv
13
results/m4X/m4X_13.csv
14
results/m4X/m4X_14.csv
15
results/m4X/m4X_15.csv
16
results/m4X/m4X_16.csv
17
results/m4X/m4X_17.csv
18
results/m4X/m4X_18.csv
19
results/m4X/m4X_19.csv
20
results/m4X/m4X_20.csv
21
results/m4X/m4X_21.csv
22
results/m4X/m4X_22.csv
23
results/m4X/m4X_23.csv
24
results/m4X/m4X_24.csv
25
results/m4X/m4X_25.csv
26
results/m4X/m4X_26.csv
27
results/m4X/m4X_27.csv
28
results/m4X/m4X_28.csv
29
results/m4X/m4X_29.csv
30
results/m4X/m4X_30.csv
31
results/m4X/m4X_31.csv
32
results/m4X/m4X_32.csv
33
results/m4X/m4X_33.csv
34
results/m4X/m4X_34.csv
35
results/m4X/m4X_35.csv
36
results/m4X/m4X_36.csv
37
results/m4X/m4X_37.csv
38
results/m4X/m4X_38.csv
39
res

In [24]:
np.random.seed(725)
random.seed(725)
prep_g = funcion_preparadora(g_m5x)

nomen = "m5X"
pathnomen = "results/" + nomen
Path(pathnomen).mkdir(parents=True, exist_ok=True)


for i in range(0, 100):
    print(i)
    np.random.seed(i)
    random.seed(i)
    outnam = pathnomen + "/" + nomen + "_" + str(i) + ".csv"
    print(outnam)
    mod_nuloAbs = EoN.Gillespie_simple_contagion(G=prep_g, 
                                            spontaneous_transition_graph=H,
                                            nbr_induced_transition_graph=J, 
                                            IC=IC, 
                                            return_statuses=return_statuses,
                                            return_full_data=True,
                                            tmax=1000
                                            )
    my_result = mod_nuloAbs
    my_result.summary()[0]
    predata = my_result.summary()[1]
    predata["t"] = my_result.summary()[0]
    df = pd.DataFrame(predata)
    df = df.assign(I_pc=100*df['I']/poblacion_escalada)
    df = df.assign(R_pc=100*df['R']/poblacion_escalada)
    df.to_csv(outnam)

    

0
results/m5X/m5X_0.csv
1
results/m5X/m5X_1.csv
2
results/m5X/m5X_2.csv
3
results/m5X/m5X_3.csv
4
results/m5X/m5X_4.csv
5
results/m5X/m5X_5.csv
6
results/m5X/m5X_6.csv
7
results/m5X/m5X_7.csv
8
results/m5X/m5X_8.csv
9
results/m5X/m5X_9.csv
10
results/m5X/m5X_10.csv
11
results/m5X/m5X_11.csv
12
results/m5X/m5X_12.csv
13
results/m5X/m5X_13.csv
14
results/m5X/m5X_14.csv
15
results/m5X/m5X_15.csv
16
results/m5X/m5X_16.csv
17
results/m5X/m5X_17.csv
18
results/m5X/m5X_18.csv
19
results/m5X/m5X_19.csv
20
results/m5X/m5X_20.csv
21
results/m5X/m5X_21.csv
22
results/m5X/m5X_22.csv
23
results/m5X/m5X_23.csv
24
results/m5X/m5X_24.csv
25
results/m5X/m5X_25.csv
26
results/m5X/m5X_26.csv
27
results/m5X/m5X_27.csv
28
results/m5X/m5X_28.csv
29
results/m5X/m5X_29.csv
30
results/m5X/m5X_30.csv
31
results/m5X/m5X_31.csv
32
results/m5X/m5X_32.csv
33
results/m5X/m5X_33.csv
34
results/m5X/m5X_34.csv
35
results/m5X/m5X_35.csv
36
results/m5X/m5X_36.csv
37
results/m5X/m5X_37.csv
38
results/m5X/m5X_38.csv
39
res

### Smart module selection
Here, we pick small modules... 

In [25]:
#20% population using a smart module selection
g_smartmod_01 = nx.convert_node_labels_to_integers(nx.read_graphml("results/redes_eon/g_reapertura_smartmod_01.graphml")) #2X

In [26]:
np.random.seed(725)
random.seed(725)
prep_g = funcion_preparadora(g_smartmod_01)

nomen = "smartmod_01"
pathnomen = "results/" + nomen
Path(pathnomen).mkdir(parents=True, exist_ok=True)


for i in range(0, 100):
    print(i)
    np.random.seed(i)
    random.seed(i)
    outnam = pathnomen + "/" + nomen + "_" + str(i) + ".csv"
    print(outnam)
    mod_nuloAbs = EoN.Gillespie_simple_contagion(G=prep_g, 
                                            spontaneous_transition_graph=H,
                                            nbr_induced_transition_graph=J, 
                                            IC=IC, 
                                            return_statuses=return_statuses,
                                            return_full_data=True,
                                            tmax=1000
                                            )
    my_result = mod_nuloAbs
    my_result.summary()[0]
    predata = my_result.summary()[1]
    predata["t"] = my_result.summary()[0]
    df = pd.DataFrame(predata)
    df = df.assign(I_pc=100*df['I']/poblacion_escalada)
    df = df.assign(R_pc=100*df['R']/poblacion_escalada)
    df.to_csv(outnam)

    

0
results/smartmod_01/smartmod_01_0.csv
1
results/smartmod_01/smartmod_01_1.csv
2
results/smartmod_01/smartmod_01_2.csv
3
results/smartmod_01/smartmod_01_3.csv
4
results/smartmod_01/smartmod_01_4.csv
5
results/smartmod_01/smartmod_01_5.csv
6
results/smartmod_01/smartmod_01_6.csv
7
results/smartmod_01/smartmod_01_7.csv
8
results/smartmod_01/smartmod_01_8.csv
9
results/smartmod_01/smartmod_01_9.csv
10
results/smartmod_01/smartmod_01_10.csv
11
results/smartmod_01/smartmod_01_11.csv
12
results/smartmod_01/smartmod_01_12.csv
13
results/smartmod_01/smartmod_01_13.csv
14
results/smartmod_01/smartmod_01_14.csv
15
results/smartmod_01/smartmod_01_15.csv
16
results/smartmod_01/smartmod_01_16.csv
17
results/smartmod_01/smartmod_01_17.csv
18
results/smartmod_01/smartmod_01_18.csv
19
results/smartmod_01/smartmod_01_19.csv
20
results/smartmod_01/smartmod_01_20.csv
21
results/smartmod_01/smartmod_01_21.csv
22
results/smartmod_01/smartmod_01_22.csv
23
results/smartmod_01/smartmod_01_23.csv
24
results/s

### Comparar reapertura smartmod vs not smart mod al 50% de la población

In [27]:
#20% population using a smart module selection
g_yessmartmod_50 = nx.convert_node_labels_to_integers(nx.read_graphml("results/redes_eon/g_reapertura_yessmart_mod50.graphml")) 
g_notsmartmod_50 = nx.convert_node_labels_to_integers(nx.read_graphml("results/redes_eon/g_reapertura_nonsmart_mod50.graphml")) 

In [28]:
np.random.seed(725)
random.seed(725)
prep_g = funcion_preparadora(g_yessmartmod_50)

nomen = "smartmod_50"
pathnomen = "results/" + nomen
Path(pathnomen).mkdir(parents=True, exist_ok=True)


for i in range(0, 100):
    print(i)
    np.random.seed(i)
    random.seed(i)
    outnam = pathnomen + "/" + nomen + "_" + str(i) + ".csv"
    print(outnam)
    mod_nuloAbs = EoN.Gillespie_simple_contagion(G=prep_g, 
                                            spontaneous_transition_graph=H,
                                            nbr_induced_transition_graph=J, 
                                            IC=IC, 
                                            return_statuses=return_statuses,
                                            return_full_data=True,
                                            tmax=1000
                                            )
    my_result = mod_nuloAbs
    my_result.summary()[0]
    predata = my_result.summary()[1]
    predata["t"] = my_result.summary()[0]
    df = pd.DataFrame(predata)
    df = df.assign(I_pc=100*df['I']/poblacion_escalada)
    df = df.assign(R_pc=100*df['R']/poblacion_escalada)
    df.to_csv(outnam)

    

0
results/smartmod_50/smartmod_50_0.csv
1
results/smartmod_50/smartmod_50_1.csv
2
results/smartmod_50/smartmod_50_2.csv
3
results/smartmod_50/smartmod_50_3.csv
4
results/smartmod_50/smartmod_50_4.csv
5
results/smartmod_50/smartmod_50_5.csv
6
results/smartmod_50/smartmod_50_6.csv
7
results/smartmod_50/smartmod_50_7.csv
8
results/smartmod_50/smartmod_50_8.csv
9
results/smartmod_50/smartmod_50_9.csv
10
results/smartmod_50/smartmod_50_10.csv
11
results/smartmod_50/smartmod_50_11.csv
12
results/smartmod_50/smartmod_50_12.csv
13
results/smartmod_50/smartmod_50_13.csv
14
results/smartmod_50/smartmod_50_14.csv
15
results/smartmod_50/smartmod_50_15.csv
16
results/smartmod_50/smartmod_50_16.csv
17
results/smartmod_50/smartmod_50_17.csv
18
results/smartmod_50/smartmod_50_18.csv
19
results/smartmod_50/smartmod_50_19.csv
20
results/smartmod_50/smartmod_50_20.csv
21
results/smartmod_50/smartmod_50_21.csv
22
results/smartmod_50/smartmod_50_22.csv
23
results/smartmod_50/smartmod_50_23.csv
24
results/s

In [29]:
np.random.seed(725)
random.seed(725)
prep_g = funcion_preparadora(g_notsmartmod_50)

nomen = "m50"
pathnomen = "results/" + nomen
Path(pathnomen).mkdir(parents=True, exist_ok=True)


for i in range(0, 100):
    print(i)
    np.random.seed(i)
    random.seed(i)
    outnam = pathnomen + "/" + nomen + "_" + str(i) + ".csv"
    print(outnam)
    mod_nuloAbs = EoN.Gillespie_simple_contagion(G=prep_g, 
                                            spontaneous_transition_graph=H,
                                            nbr_induced_transition_graph=J, 
                                            IC=IC, 
                                            return_statuses=return_statuses,
                                            return_full_data=True,
                                            tmax=1000
                                            )
    my_result = mod_nuloAbs
    my_result.summary()[0]
    predata = my_result.summary()[1]
    predata["t"] = my_result.summary()[0]
    df = pd.DataFrame(predata)
    df = df.assign(I_pc=100*df['I']/poblacion_escalada)
    df = df.assign(R_pc=100*df['R']/poblacion_escalada)
    df.to_csv(outnam)

    

0
results/m50/m50_0.csv
1
results/m50/m50_1.csv
2
results/m50/m50_2.csv
3
results/m50/m50_3.csv
4
results/m50/m50_4.csv
5
results/m50/m50_5.csv
6
results/m50/m50_6.csv
7
results/m50/m50_7.csv
8
results/m50/m50_8.csv
9
results/m50/m50_9.csv
10
results/m50/m50_10.csv
11
results/m50/m50_11.csv
12
results/m50/m50_12.csv
13
results/m50/m50_13.csv
14
results/m50/m50_14.csv
15
results/m50/m50_15.csv
16
results/m50/m50_16.csv
17
results/m50/m50_17.csv
18
results/m50/m50_18.csv
19
results/m50/m50_19.csv
20
results/m50/m50_20.csv
21
results/m50/m50_21.csv
22
results/m50/m50_22.csv
23
results/m50/m50_23.csv
24
results/m50/m50_24.csv
25
results/m50/m50_25.csv
26
results/m50/m50_26.csv
27
results/m50/m50_27.csv
28
results/m50/m50_28.csv
29
results/m50/m50_29.csv
30
results/m50/m50_30.csv
31
results/m50/m50_31.csv
32
results/m50/m50_32.csv
33
results/m50/m50_33.csv
34
results/m50/m50_34.csv
35
results/m50/m50_35.csv
36
results/m50/m50_36.csv
37
results/m50/m50_37.csv
38
results/m50/m50_38.csv
39
res